# Reserve and configure FABRIC resources for "setting intermediate network"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [5]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [6]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [7]:
slice_name="intermediate-network-8"
slice = fablib.new_slice(name=slice_name)

We will now set up all the parameters required to realize the network:

In [119]:
import random
#number of routers in access network similar irrespective of cdn or not
#redundancy_AS_1 parameter sets the hop at which redundant paths will be set
hops_AS_1=random.randint(3,6)
redundancy_AS_1=3

# For cdn, define
# number of autonomous systems including source and target AS (ranges between 2-4)
# hops seen in destination network (ranges between 3-4)
# hops seen in intermediate network (ranges bewteen 1-2)

number_of_AS_cdn=random.randint(2,4)
hops_final_AS_cdn=random.randint(3,4)
max_hops_intermediate_cdn=3

# For non CDN, define
# number of autonomous systems including source and target AS (ranges between 3-5)
# hops seen in destination network (ranges between 5-8)
# hops seen in intermediate network (ranges bewteen 1-4)

number_of_AS_not_cdn=random.randint(3,5)
hops_final_AS_not_cdn=random.randint(5,8)
max_hops_intermediate_not_cdn=4

# Set cdn=True if CDN type opf network is required else False
cdn=True
sites=[]
total_routers=[]
routers_conf=[]
routers_net=[]



In [120]:
# Print all the parameters:
print("Hops in first AS: ", hops_AS_1)
print("Number of AS: ", number_of_AS_cdn if cdn else number_of_AS_not_cdn)
print("Hops in final AS: ", hops_final_AS_cdn if cdn else hops_final_AS_not_cdn )


Hops in first AS:  4
Number of AS:  4
Hops in final AS:  3


In [121]:
# Set up the routers and networks in all the autonomous systems
if cdn:
    number_of_AS=number_of_AS_cdn
    hops_final_AS=hops_final_AS_cdn
    max_hops_intermediate=max_hops_intermediate_cdn
    
else:    
    number_of_AS=number_of_AS_not_cdn
    hops_final_AS=hops_final_AS_not_cdn
    max_hops_intermediate=max_hops_intermediate_not_cdn


# set up the number of routers in first AS

data_routers=[create_AS(0, hops_AS_1, redundancy_AS_1)]

  
# set up the number of routers in intermediate AS
# hops in intermediate AS is randomly selected between 1 and max_hops_intermediate value
# redundant path can start at any value between 2nd hop and hops_intermediate-1 in the intermediate AS

for i in range(1, number_of_AS-1):
    hops_intermediate=random.randint(1,max_hops_intermediate)
    print(hops_intermediate)
    data_routers+=[create_AS(i, hops_intermediate, r=random.randint(1,hops_intermediate))]

# set up the number of routers in Destination AS
# redundant path can start at any value between 2nd hop and hops_final_AS-1 in the destination AS
data_routers+=[create_AS(number_of_AS-1, hops_final_AS, random.randint(1,hops_final_AS))]

#print the routers, configuration and network data
total_routers+=[data_routers[i][0] for i in range(number_of_AS)]
routers_conf+=[data_routers[i][1] for i in range(number_of_AS)]
routers_net+=[data_routers[i][2] for i in range(number_of_AS)]
print(total_routers)


0 4 3
1
1 1 1
1
2 1 1
3 3 2
[[1, 1, 1, 1], [1], [1], [1, 1, 1]]


In [118]:
# For dedicated NIC between each router pair (subnet and IP address rangs from 1 to number of networks )
def create_AS(as_num, n_hops, r):
    print(as_num, n_hops, r)
    routers=[1]*min(r, n_hops) + [random.randint(2,3) for i in range(max(0, n_hops - r - 1))] + [1 for i in range(0,1) if n_hops!=r]
    node_conf = [ {'name': "as" + str(as_num) + '-r-' + str(j) + '-' + str(i) ,
        'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}
         for j,level in enumerate(routers) for i in range(level)]
    r_mul=[0]+[routers[i-1]*routers[i] for i in range(1,len(routers))]
    net_conf = [
     {"name": "as"+str(as_num)+"-net"+str(j)+str(i)+"-"+str(j+1)+str(k), "subnet": "10."+str(as_num)+"."+str(sum(r_mul[:j+1])+(routers[j+1]*i)+k+1)+".0/24", 
      "nodes": [{"name": "as"+str(as_num)+'-r-' + str(j) + '-' + str(i),   "addr": "10."+str(as_num)+"."+str(sum(r_mul[:j+1])+(routers[j+1]*i)+k+1)+"."+str(1)} ]+ 
      [{"name": "as"+str(as_num)+'-r-' + str(j+1) + '-' + str(k),   "addr": "10."+str(as_num)+"."+str(sum(r_mul[:j+1])+(routers[j+1]*i)+k+1)+"."+str(2)}]}
            for j,level in enumerate(routers[:-1]) for i in range(level) for k in range(routers[j+1])]
    return [routers, node_conf, net_conf]


This cell will try to identify a site that has sufficient resources for the experiment. 

In [12]:
# Get the sites
for i,r in enumerate(total_routers):   
    exp_requires = {'core': (sum([(routers_conf[i][j]["cores"]) for j in range(len(routers_conf[i])) ])), 
                    'smart_nic_1': (sum([len(routers_net[i][j]["nodes"]) for j in range(len(routers_net[i])) ]))}
    print(exp_requires)
    while True:
        site_name = fablib.get_random_site(avoid=sites)
        if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
            (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['smart_nic_1']) ):
            break

    fablib.show_site(site_name)
    sites.append(site_name)
    

{'core': 32, 'smart_nic_1': 6}


Name,MASS
State,Active
Address,"100 Bigelow Street,Holyoke, MA 01040"
Location,"(42.202493000000004, -72.60787662257826)"
Hosts,3
CPUs,6
Cores Available,314
Cores Capacity,320
Cores Allocated,6
RAM Available,1478
RAM Capacity,1506


{'core': 24, 'smart_nic_1': 4}


Name,NCSA
State,Active
Address,"1725 S Oak St.,Champaign, IL 61820"
Location,"(40.1035624, -88.2415105)"
Hosts,3
CPUs,6
Cores Available,352
Cores Capacity,384
Cores Allocated,32
RAM Available,1398
RAM Capacity,1506


{'core': 16, 'smart_nic_1': 2}


Name,GATECH
State,Active
Address,"760 West Peachtree Street NW,Atlanta, GA 30308"
Location,"(33.7753991, -84.3875488)"
Hosts,5
CPUs,10
Cores Available,634
Cores Capacity,640
Cores Allocated,6
RAM Available,2492
RAM Capacity,2510


{'core': 40, 'smart_nic_1': 12}


Name,WASH
State,Active
Address,"1755 Old Meadow Road,McLean, VA 22102"
Location,"(38.9208836, -77.21119737837294)"
Hosts,3
CPUs,6
Cores Available,356
Cores Capacity,384
Cores Allocated,28
RAM Available,1462
RAM Capacity,1506


In [13]:
print(sites)

['', 'MASS', 'NCSA', 'GATECH', 'WASH']


In [14]:
for i,r in enumerate(routers_conf):
     for j in range(len(r)):
        slice.add_node(name=r[j]['name'], site=sites[i+1], 
                       cores=r[j]['cores'], 
                       ram=r[j]['ram'], 
                       disk=r[j]['disk'], 
                       image=r[j]['image'])


In [15]:
print(slice.list_nodes())

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
None,as0-r-0-0,None,None,None,default_ubuntu_22,qcow2,None,MASS,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-1-0,None,None,None,default_ubuntu_22,qcow2,None,MASS,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-2-0,None,None,None,default_ubuntu_22,qcow2,None,MASS,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-3-0,None,None,None,default_ubuntu_22,qcow2,None,MASS,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as1-r-0-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as1-r-1-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as1-r-2-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as2-r-0-0,None,None,None,default_ubuntu_22,qcow2,None,GATECH,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as2-r-1-0,None,None,None,default_ubuntu_22,qcow2,None,GATECH,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as3-r-0-0,None,None,None,default_ubuntu_22,qcow2,None,WASH,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


In [16]:
# this cell sets up the network links 
for i,r in enumerate(routers_net):
     for j in range(len(r)):
        ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                     name=r[j]["name"]).get_interfaces()[0] for node in r[j]['nodes'] ]
        slice.add_l2network(name=r[j]["name"], type='L2Bridge', interfaces=ifaces)


When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [17]:
slice.submit()


Retry: 19, Time: 1561 sec


ID,aaaf84f4-b2a8-46f1-900f-01756ddcf5c5
Name,intermediate-network-8
Lease Expiration (UTC),2023-07-27 03:20:24 +0000
Lease Start (UTC),2023-07-26 03:20:25 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
c3ecb218-40f4-41b6-94d1-df647adbd45f,as0-r-0-0,8,16,100,default_ubuntu_22,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.73,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.73,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d19393d2-0385-4f98-b71a-f72274c88bd6,as0-r-1-0,8,16,100,default_ubuntu_22,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.94,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.94,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
4065bcd7-d99f-4821-b648-359b6a8c4b2e,as0-r-2-0,8,16,100,default_ubuntu_22,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d00f6cb6-7f42-42e5-b535-2a5f0cc5ff07,as0-r-3-0,8,16,100,default_ubuntu_22,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.110,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.110,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5026a5a0-c166-4d5d-bb0e-80d152d22bb2,as1-r-0-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:feb3:8ad,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feb3:8ad,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f076eea8-17e8-48c9-991e-84068e4e7c50,as1-r-1-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fecc:c348,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fecc:c348,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
16f0d0cb-0147-45dc-9964-03fc25dd707a,as1-r-2-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe8f:a37,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe8f:a37,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
9c5dda14-1370-4f11-b574-599e83d551f4,as2-r-0-0,8,16,100,default_ubuntu_22,qcow2,gatech-w4.fabric-testbed.net,GATECH,ubuntu,2610:148:1f00:9f01:f816:3eff:fe5b:6867,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:148:1f00:9f01:f816:3eff:fe5b:6867,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1abc4efc-7197-407f-b83c-02b6a7d43897,as2-r-1-0,8,16,100,default_ubuntu_22,qcow2,gatech-w4.fabric-testbed.net,GATECH,ubuntu,2610:148:1f00:9f01:f816:3eff:fe56:6f2c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:148:1f00:9f01:f816:3eff:fe56:6f2c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a957a7a2-5ce7-4606-b061-716a438b3139,as3-r-0-0,8,16,100,default_ubuntu_22,qcow2,wash-w3.fabric-testbed.net,WASH,ubuntu,2001:400:a100:3020:f816:3eff:fe6c:9ad4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe6c:9ad4,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
8ba2c8fc-efd0-4a58-8030-bc661d95c16e,as0-net00-10,L2,L2Bridge,MASS,None,None,Active,
57097a0a-980e-4033-bb58-d1975cf73558,as0-net10-20,L2,L2Bridge,MASS,None,None,Active,
43ef003f-deb3-4b89-b1d6-a774ad709e28,as0-net20-30,L2,L2Bridge,MASS,None,None,Active,
a3e53a4c-281d-43e9-831e-53c64f3a9ecc,as1-net00-10,L2,L2Bridge,NCSA,None,None,Active,
1ccb1b6e-7216-42bc-a5fa-bf7be03280e2,as1-net10-20,L2,L2Bridge,NCSA,None,None,Active,
04dcf615-2337-440d-a52d-e61842bcd666,as2-net00-10,L2,L2Bridge,GATECH,None,None,Active,
6c29a846-1ff2-4f50-b083-7b9791b6bf86,as3-net00-10,L2,L2Bridge,WASH,None,None,Active,
fba8b48c-5ea1-4a28-a107-049b05d4fce1,as3-net00-11,L2,L2Bridge,WASH,None,None,Active,
49145dd6-e0ca-4941-ad53-6e65c164b997,as3-net00-12,L2,L2Bridge,WASH,None,None,Active,
a967a482-5803-4ac8-a632-830eb9ddb962,as3-net10-20,L2,L2Bridge,WASH,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
as0-r-0-0-as0-net00-10-p1,p1,as0-r-0-0,as0-net00-10,100,config,,02:ED:62:77:0C:1F,ens7,ens7,None,4
as0-r-1-0-as0-net00-10-p1,p1,as0-r-1-0,as0-net00-10,100,config,,0E:07:0F:DC:B6:BF,ens8,ens8,None,4
as0-r-1-0-as0-net10-20-p1,p1,as0-r-1-0,as0-net10-20,100,config,,06:1F:06:7D:4A:1A,ens7,ens7,None,4
as0-r-2-0-as0-net10-20-p1,p1,as0-r-2-0,as0-net10-20,100,config,,0E:9A:31:A3:8D:B4,ens7,ens7,None,4
as0-r-2-0-as0-net20-30-p1,p1,as0-r-2-0,as0-net20-30,100,config,,12:99:13:E2:E0:8C,ens8,ens8,None,4
as0-r-3-0-as0-net20-30-p1,p1,as0-r-3-0,as0-net20-30,100,config,,16:04:94:39:5F:01,ens7,ens7,None,4
as1-r-0-0-as1-net00-10-p1,p1,as1-r-0-0,as1-net00-10,100,config,,12:53:E3:FD:D0:2E,ens7,ens7,None,4
as1-r-1-0-as1-net00-10-p1,p1,as1-r-1-0,as1-net00-10,100,config,,12:B2:D2:8A:4B:71,ens8,ens8,None,4
as1-r-1-0-as1-net10-20-p1,p1,as1-r-1-0,as1-net10-20,100,config,,12:A6:83:DE:92:B0,ens7,ens7,None,4
as1-r-2-0-as1-net10-20-p1,p1,as1-r-2-0,as1-net10-20,100,config,,12:9B:F7:5D:64:BC,ens7,ens7,None,4



Time to print interfaces 1592 seconds


'aaaf84f4-b2a8-46f1-900f-01756ddcf5c5'

## Configure resources

In [18]:
slice.get_state()

'StableOK'

In [19]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

Bring up all of the network interfaces:

In [21]:
from ipaddress import ip_address, IPv4Address, IPv4Network

for i,r in enumerate(routers_net):
     for j in range(len(r)):
        for node in r[j]['nodes']:
            if_name = node['name'] + '-' + r[j]['name'] + '-p1'
            iface = slice.get_interface(if_name)
            iface.ip_link_up()
            #iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())
            iface.ip_addr_add(addr=node['addr'], subnet=IPv4Network(r[j]['subnet']))



In [22]:
router_nodes=[]
for i,r in enumerate(routers_conf):
    nodes=[slice.get_node(name=r[j]['name']) for j in range(len(r))]
    router_nodes.append(nodes)
    

In [23]:
for i,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n.get_name())
        n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
        n.execute("sudo apt-get update; sudo apt-get -y install net-tools", quiet=True)
        n.execute("wget -O - https://git.io/JYhs5 | bash")
    

as0-r-0-0
net.ipv4.ip_forward = 1
OK
 --2023-07-26 03:51:11--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/ffund/59b15ab7700e9602813e7c096cda9e13/raw/63a0ab127d34a168924aa6fc61308a36bd06ddcf/multicast-router-install.sh [following]
--2023-07-26 03:51:12--  https://gist.githubusercontent.com/ffund/59b15ab7700e9602813e7c096cda9e13/raw/63a0ab127d34a168924aa6fc61308a36bd06ddcf/multicast-router-install.sh
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391 [text/plain]
Saving to: ‘STDOUT’

     0K                                                       100% 56.6M=0s

2023-07-

In [24]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n)
        n.execute("ifconfig -a")

-----------------  --------------------------------------------------------------------------------------------------------------------
ID                 c3ecb218-40f4-41b6-94d1-df647adbd45f
Name               as0-r-0-0
Cores              8
RAM                16
Disk               100
Image              default_ubuntu_22
Image Type         qcow2
Host               mass-w2.fabric-testbed.net
Site               MASS
Management IP      205.172.170.73
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.73
-----------------  --------------------------------------------------------------------------------------------------------------------
ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
        inet 10.20.5.178  netmask 255.255.254.0  broadcast 10.20.5.255
        inet6 fe80::f816:3eff:feff:7616  prefixlen 64  scopeid 0x20<link>
        ether fa:16:3e:ff:76:16

In [27]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c'configure terminal\nrouter ospf\nnetwork 10.0.0.0/8 area 0.0.0.0\n exit\n exit\n exit'  ")


Hello, this is FRRouting (version 8.5.2).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

as0-r-0-0# 
as0-r-0-0# configure terminal
as0-r-0-0(config)# router ospf
as0-r-0-0(config-router)# network 10.0.0.0/8 area 0.0.0.0
as0-r-0-0(config-router)#  exit
as0-r-0-0(config)#  exit
as0-r-0-0#  exit

Hello, this is FRRouting (version 8.5.2).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

as0-r-1-0# 
as0-r-1-0# configure terminal
as0-r-1-0(config)# router ospf
as0-r-1-0(config-router)# network 10.0.0.0/8 area 0.0.0.0
as0-r-1-0(config-router)#  exit
as0-r-1-0(config)#  exit
as0-r-1-0#  exit

Hello, this is FRRouting (version 8.5.2).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

as0-r-2-0# 
as0-r-2-0# configure terminal
as0-r-2-0(config)# router ospf
as0-r-2-0(config-router)# network 10.0.0.0/8 area 0.0.0.0
as0-r-2-0(config-router)#  exit
as0-r-2-0(config)#  exit
as0-r-2-0#  exit

Hello, this is FRRouting (version 8.5.2).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

as0-r-3-0# 
as0-r-3-0# co

In [28]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c 'show ip route\nexit'")


Hello, this is FRRouting (version 8.5.2).
Copyright 1996-2005 Kunihiro Ishiguro, et al.

as0-r-0-0# 
as0-r-0-0# show ip route
Codes: K - kernel route, C - connected, S - static, R - RIP,
       O - OSPF, I - IS-IS, B - BGP, E - EIGRP, N - NHRP,
       T - Table, v - VNC, V - VNC-Direct, A - Babel, F - PBR,
       f - OpenFabric,
       > - selected route, * - FIB route, q - queued, r - rejected, b - backup
       t - trapped, o - offload failure

K>* 0.0.0.0/0 [0/100] via 10.20.4.1, ens3, src 10.20.5.178, 00:15:52
O   10.0.1.0/24 [110/1] is directly connected, ens7, weight 1, 00:00:27
C>* 10.0.1.0/24 is directly connected, ens7, 00:15:52
O   10.20.4.0/23 [110/1] is directly connected, ens3, weight 1, 00:00:27
C>* 10.20.4.0/23 [0/100] is directly connected, ens3, 00:15:52
K>* 10.20.4.1/32 [0/100] is directly connected, ens3, 00:15:52
K>* 128.163.180.135/32 [0/100] via 10.20.4.1, ens3, src 10.20.5.178, 00:15:52
K>* 152.54.15.5/32 [0/100] via 10.20.4.1, ens3, src 10.20.5.178, 00:15:52
K>

In [ ]:
# slice.delete()

In [29]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 4 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)